# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [19]:
import os
import openai
from utils import format_message, show_prompt


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# Set the model variable based on the current date
llm_model = "gpt-4o-mini"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [3]:

client = openai.OpenAI()

def get_completion(prompt, model=llm_model):
    messages = [
        {
        "role": "user",
        "content": prompt
        }
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content


In [15]:
client = openai.OpenAI()

with client.chat.completions.stream(
    model=llm_model,
    messages=[{"role": "user", "content": "Explique brevemente o que é aprendizado de máquina."}],
    temperature=0,
) as stream:
    for event in stream:
        if event.type == "message.delta":
            print(event.delta, end="", flush=True)

In [ ]:
response = get_completion("Explique brevemente o que é aprendizado de máquina.")
print(response)

Aprendizado de máquina é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados. Em vez de serem programados explicitamente para realizar uma tarefa, os sistemas de aprendizado de máquina identificam padrões e fazem previsões ou decisões com base em exemplos fornecidos. Isso é feito através de técnicas como regressão, classificação e agrupamento, e é amplamente utilizado em diversas aplicações, como reconhecimento de voz, recomendação de produtos, detecção de fraudes e muito mais. O objetivo é melhorar o desempenho do modelo à medida que mais dados se tornam disponíveis.


In [9]:
styles = ["formal and technical", "casual and friendly", "enthusiastic and persuasive", "concise and to the point", "storytelling and engaging"]
tones = ["confident", "empathetic", "urgent", "optimistic", "serious"]

In [26]:
prompt_task = """Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.

<Task>
Redigir um email com base no contexto fornecido.
</Task>

O email precisa ter a seguinte estrutura básica:
<Structure>
1. Saudação
2. Introdução ao problema
3. Apresentação da solução
4. Benefícios da solução
5. Chamada para ação (CTA)
6. Despedida
</Structure>

"""
prompt_context = """
<Context>
Escreva um email para um cliente corporativo com o seguinte contexto:
{customer_email}
O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.
</Context>
"""

prompt_instructions = """
<Instrutions>
Use o estilo e tom especificados para redigir o email.
Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.
Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.
Não invente informações; baseie-se apenas no contexto fornecido.
Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.
</Instrutions>
"""

prompt_references = """
<Example>
Por exemplo, um email formal e técnico com tom confiante pode ser:
Asunto: Apresentação de Solução Avançada de Visão Computacional para Inspeção Industrial
Prezado Sr. Silva,
Gostaria de apresentar nossa avançada solução de visão computacional projetada para otimizar processos de inspeção industrial.
Nossa tecnologia utiliza algoritmos de ponta para garantir precisão e eficiência, reduzindo custos operacionais.
Ficaria honrado em agendar uma demonstração para discutir como nossa solução pode beneficiar sua empresa.
Atenciosamente,
João Pereira
</Example>
"""

prompt = f"""{prompt_task}
{prompt_context}
{prompt_instructions}
{prompt_references}
"""


In [27]:

show_prompt(prompt)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

In [30]:
customer_email = """
Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.
"""

print(customer_email)


Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.



In [31]:
from datetime import datetime
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

prompt.format(customer_email=customer_email, style=styles[0], tone=tones[0], date=get_today_str())
response = get_completion(prompt)

In [33]:
print(response)

Assunto: Proposta de Solução para Otimização de Processos

Prezado [Nome do Cliente],

Espero que este email o encontre bem.

Identificamos que sua empresa enfrenta desafios na otimização de processos, o que pode impactar a eficiência e a produtividade. Sabemos que, em um mercado competitivo, é crucial encontrar maneiras de melhorar continuamente.

Para abordar essa questão, gostaríamos de apresentar nossa solução inovadora de automação de processos. Nossa plataforma foi desenvolvida para integrar-se facilmente aos sistemas existentes, proporcionando uma gestão mais eficiente e reduzindo o tempo gasto em tarefas manuais.

Os benefícios dessa solução incluem:
- Aumento significativo na eficiência operacional.
- Redução de erros humanos e retrabalho.
- Melhoria na alocação de recursos, permitindo que sua equipe se concentre em atividades estratégicas.

Estamos confiantes de que nossa solução pode trazer resultados positivos para sua empresa. Gostaríamos de agendar uma demonstração para q

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [25]:
#!pip install --upgrade langchain
#!pip install langchain_openai

### Model

In [34]:
from langchain_openai import ChatOpenAI

In [35]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    temperature=0.0,
    model=llm_model
    )
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x79600393efd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x79600393ff10>, root_client=<openai.OpenAI object at 0x79600393ed50>, root_async_client=<openai.AsyncOpenAI object at 0x79600393f9d0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_organization='org-5qZfsmr4Uz6f4Olo3UqM4Fgb', stream_usage=True)

In [36]:
from langchain_core.messages import HumanMessage

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = chat.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [37]:
from utils import format_message, show_prompt

show_prompt(prompt)
format_message([response])

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Assunto: Proposta de Solução para Otimização de Processos                                                       │
│                                                                                                                 │
│ Prezado [Nome do Cliente],                                                                                      │
│                                                                                                                 │
│ Espero que este email o encontre bem. Estou entrando em contato para discutir um desafio que muitas empresas    │
│ enfrentam atualmente: a necessidade de otimizar processos para aumentar a eficiência e reduzir custos.          │
│                                                                                                                 │
│ Para abordar essa questão, nossa equipe desenvolveu uma solução inovadora que integra tecnologia de automação e │
│ análise de dados. Essa ferramenta não apenas simplifica os fluxos de trabalho, mas também proporciona insights  │
│ valiosos que podem impulsionar a tomada de decisões estratégicas.                                               │
│                                                                                                                 │
│ Os benefícios dessa solução são significativos: aumento da produtividade, redução de erros operacionais e,      │
│ consequentemente, uma economia considerável nos custos. Além disso, a implementação é rápida e o suporte        │
│ contínuo da nossa equipe garante uma transição suave.                                                           │
│                                                                                                                 │
│ Gostaria de convidá-lo para agendar uma demonstração, onde poderemos explorar como essa solução pode ser        │
│ adaptada às necessidades específicas da sua empresa. Por favor, me avise sobre sua disponibilidade para que     │
│ possamos marcar um horário conveniente.                                                                         │
│                                                                                                                 │
│ Agradeço pela atenção e fico à disposição para qualquer dúvida.                                                 │
│                                                                                                                 │
│ Atenciosamente,                                                                                                 │
│                                                                                                                 │
│ [Seu Nome]                                                                                                      │
│ [Seu Cargo]                                                                                                     │
│ [Nome da Empresa]                                                                                               │
│ [Telefone]                                                                                                      │
│ [Email]                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Format output

In [ ]:
from pydantic import BaseModel, Field

class EmailResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    tips: str = Field(..., description="Additional tips for improving the email.")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")


In [41]:
email_writer_chain = (
    chat
    .with_structured_output(EmailResponse)
)

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = email_writer_chain.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [45]:
response

EmailResponse(customer_email='Prezado Sr. Oliveira,\n\nEspero que esteja bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de otimizar a gestão de projetos e aumentar a produtividade das equipes.\n\nPara isso, gostaria de apresentar nossa plataforma de gerenciamento de projetos, que oferece ferramentas integradas para planejamento, acompanhamento e colaboração em tempo real. Com nossa solução, sua equipe poderá visualizar o progresso das tarefas, identificar gargalos e priorizar atividades de forma mais eficiente.\n\nOs benefícios são claros: aumento da transparência nas operações, melhor alocação de recursos e, consequentemente, um aumento significativo na produtividade. Além disso, nossa plataforma é intuitiva e fácil de usar, o que facilita a adoção por parte de todos os membros da equipe.\n\nGostaria de agendar uma demonstração para que você possa ver em primeira mão como nossa solução pode transformar a gestão de projet

In [ ]:
print("Customer Email:")
print(response.customer_email)
print("\nTips:")
print(response.tips)
print("\nChain of Thought:")
print(response.chain_of_thought)

Customer Email:
Prezado Sr. Oliveira,

Espero que esteja bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de otimizar a gestão de projetos e aumentar a produtividade das equipes.

Para isso, gostaria de apresentar nossa plataforma de gerenciamento de projetos, que oferece ferramentas integradas para planejamento, acompanhamento e colaboração em tempo real. Com nossa solução, sua equipe poderá visualizar o progresso das tarefas, identificar gargalos e priorizar atividades de forma mais eficiente.

Os benefícios são claros: aumento da transparência nas operações, melhor alocação de recursos e, consequentemente, um aumento significativo na produtividade. Além disso, nossa plataforma é intuitiva e fácil de usar, o que facilita a adoção por parte de todos os membros da equipe.

Gostaria de agendar uma demonstração para que você possa ver em primeira mão como nossa solução pode transformar a gestão de projetos na sua empresa. Por

### Exercisio

Para aprimorar os resultados obtidos vamos a criar um sistemas multi-agente para conseguir fazer uma análise do problema (reflexão), escrever o email a partir deste plano (escrita) e finalmente avaliar os resultados obtidos (avaliação). 

- Step 1: Crie os três agentes (Agente reflexão, Agente escrita, Agente avaliação)
- Step 2: Crie um pipeline (chain) para gerar os resultados
- Step 3: Avalie e intere sobre os resultados obtidos 
